<h1>Hamming Distance k-NN Search with FAISS</h1>

In [1]:
# import libraries
import os
import math
import faiss
import numpy as np
#from keras import backend as K
from IPython.display import clear_output
#from keras.models import Model, load_model

Loading faiss.


<h1>Bit Vectorize Embeddings</h1>
<p>Here we convert the embeddings into bit vectors.<br>
We do this by using the following rule:<br>
Nodes > .5 = 1<br>
Nodes <= .5 = 0
</p>

In [2]:
# bit vectorize embeddings
def bit_vectorize(x, padding):
    bit_arr = []
    for i in range(len(x)):
        b_arr = []
        for j in range(len(x[i])):
            if x[i][j] > .5:
                b_arr.append(1)
            else:
                b_arr.append(0)
        b_arr = b_arr + padding
        pb = np.packbits(b_arr)
        bit_arr.append(pb)
    rarr = np.ascontiguousarray(np.array(bit_arr))
    return rarr

<h1>Set Parameters for Search</h1>

In [3]:
# configurations
# number of embeddings to generate (change load to false in next cell)
n_test = 100000
n_train = 1000000
bitsize = 50
# set padding
p = int((math.ceil(bitsize/8.0))*8 - bitsize)
padding = []
for i in range(p):
    padding.append(0)

# similarity search
bank_size = 100000
q_size = 50000
d = bitsize + len(padding)
k = 5

<h1>Option to Load or Generate Bit Vector Data</h1>

In [4]:
load_bit_vector = True
load_test = True

# get bit vectors for database
print('Loading Bank ...')
if load_bit_vector == True:
    path = 'data/bit_vector/faiss_test/bit_vector_train' + str(bitsize) + '.npy'
    bit_vectors = np.load(path)

# option to generate and save bit vectors
else:
    # load signal data
    x_train = np.load('data/npy_data/signal_dataset/train/signals.npy')
    x_train = x_train[:n_train]
    x_train = x_train.reshape([-1, 1024, 2])
    # load model
    model = load_model('weights_best/hamm' + str(bitsize) + '_weights.hdf5')
    # get output from embedding layer
    embedding_output = Model(inputs=model.input, outputs=model.get_layer('dense_3').output)
    embeddings = embedding_output.predict(x_train)
    # bit vectorize embedding
    bit_vectors = bit_vectorize(embeddings, padding)
    np.save('data/bit_vector/faiss_test/bit_vector_train' + str(bitsize) + '.npy', bit_vectors)
print('Bank Load Complete!' + '\n')

# load snr and label data for database 
print('Loading labels and snr values ...')
labels = np.load('data/npy_data/signal_dataset/train/labels.npy')
labels = labels[:n_train]
snrs = np.load('data/npy_data/signal_dataset/train/snrs.npy')
snrs = snrs[:n_train]
print('Load complete!', '\n')


# get bit vectors for query set
print('Loading Queries ...')
if load_test == True:
    path = 'data/bit_vector/faiss_test/bit_vector_test' + str(bitsize) + '.npy'
    bit_vectors_test = np.load(path)

# option to generate and save bit vectors
else:
    # load signal data
    x_test = np.load('data/npy_data/signal_dataset/test/signals.npy')
    x_test = x_test[:n_test]
    x_test = x_test.reshape([-1, 1024, 2])
    # load model
    model = load_model('weights_best/hamm' + str(bitsize) + '_weights.hdf5')
    # get output from embedding layer
    embedding_output = Model(inputs=model.input, outputs=model.get_layer('dense_3').output)
    embeddings_test = embedding_output.predict(x_test)
    # bit vectorize embedding
    bit_vectors_test = bit_vectorize(embeddings_test, padding)
    np.save('data/bit_vector/faiss_test/bit_vector_test' + str(bitsize) + '.npy', bit_vectors_test)
print('Queries Load Complete!' + '\n')

# load snr and label data for query set
print('Loading labels and snr values ...')
test_labels = np.load('data/npy_data/signal_dataset/test/labels.npy')
test_labels = test_labels[:n_test]
test_snrs = np.load('data/npy_data/signal_dataset/test/snrs.npy')
test_snrs = test_snrs[:n_test]
print('Load complete!', '\n')


Loading Bank ...
Bank Load Complete!

Loading labels and snr values ...
Load complete! 

Loading Queries ...
Queries Load Complete!

Loading labels and snr values ...
Load complete! 



<h1>Similarity Search with FAISS<h1>

In [5]:
# initialize database
db = bit_vectors[:bank_size]
print('Size of database: ', len(db))
# initialize query set
queries = bit_vectors_test[:q_size]
print('Size of query set: ', len(queries))
print('\n')
# Initializing index.
index = faiss.IndexBinaryFlat(d)
# Adding the database vectors.
index.add(db)
print('Searching for nearest neighbors ...')
D, I = index.search(queries, k)
print('Search complete!')

Size of database:  100000
Size of query set:  50000


Searching for nearest neighbors ...
Search complete!


<h1>Get Accuracy</h1>

In [6]:
# vote function
def vote(lst):
    return max(set(lst), key=lst.count)

# get accuracy 
total = dict(zip(range(-20,31), [0]*len(range(-20,31))))
total_correct = dict(zip(range(-20,31), [0]*len(range(-20,31))))
for i in range(len(I)):
    class_idx = []
    q_snr = test_snrs[i]
    total[q_snr] = total[q_snr] + 1
    for j in range(k):
        tr_idx = I[i][j]
        class_idx.append(str(labels[tr_idx]))
    if str(test_labels[i]) == str(vote(class_idx)):
        total_correct[q_snr] = total_correct[q_snr] + 1

for i in total.keys():
    if total[i] != 0:
        accuracy = round((total_correct[i]/total[i])*100, 2)
        print("Accuracy at snr = %d and k = %d: "%(i,k), accuracy)

Accuracy at snr = -20 and k = 5:  4.6
Accuracy at snr = -18 and k = 5:  4.91
Accuracy at snr = -16 and k = 5:  4.56
Accuracy at snr = -14 and k = 5:  4.81
Accuracy at snr = -12 and k = 5:  6.08
Accuracy at snr = -10 and k = 5:  9.35
Accuracy at snr = -8 and k = 5:  15.16
Accuracy at snr = -6 and k = 5:  22.86
Accuracy at snr = -4 and k = 5:  29.55
Accuracy at snr = -2 and k = 5:  37.5
Accuracy at snr = 0 and k = 5:  48.99
Accuracy at snr = 2 and k = 5:  55.03
Accuracy at snr = 4 and k = 5:  65.59
Accuracy at snr = 6 and k = 5:  75.28
Accuracy at snr = 8 and k = 5:  82.7
Accuracy at snr = 10 and k = 5:  84.75
Accuracy at snr = 12 and k = 5:  86.18
Accuracy at snr = 14 and k = 5:  84.72
Accuracy at snr = 16 and k = 5:  86.65
Accuracy at snr = 18 and k = 5:  86.14
Accuracy at snr = 20 and k = 5:  85.97
Accuracy at snr = 22 and k = 5:  85.91
Accuracy at snr = 24 and k = 5:  85.87
Accuracy at snr = 26 and k = 5:  85.36
Accuracy at snr = 28 and k = 5:  85.92
Accuracy at snr = 30 and k = 5:  

In [ ]:
# plot accuracy data
x = []
y = []
keys = list(total_correct.keys())
keys = [k for k in keys if total[k] != 0]
keys.sort()
for i in keys:
    x.append(i)
    y.append((total_correct[i]/total[i])*100)

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

#figure(figsize=(15,8))
#plt.plot(x, y)
#plt.xlabel('Signal to Noise Ratio')
#plt.ylabel('Accuracy')
#plt.show()

In [8]:
# save accuracies
#GW filename = 'data/bit_vector/snr_accuracy/hamm' + str(bitsize) + '_k' + str(k) + '.npy'
#filename = 'hamm' + str(bitsize) + '_k' + str(k) + '.npy'
#np.save(filename, y)
#print('File Saved: ' + filename)